In [ ]:
import trimesh
import numpy as np
import pandas as pd
import pymesh
import psutil
import reutils as utils
import time
import time
import relegolization1 as lego
#import reutils as utils
#import relegolization1 as lego
import fabrication as fab
import math
import networkx as nx
import debug_utils as db
from pathlib import Path
import random
import matplotlib.pyplot as plt
%matplotlib widget

The next cell creates the dataset npy array from the models dataset.

This is meant for the dataset generation but one should just use the precomputed ones

In [ ]:
single_lego_sizes = np.array([8,8,9.6])
num_reapeat = 10
num_connectivity_pair = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
resultsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "time_avg", "time_std", "bricks_avg", "bricks_std", "connected_components_avg", "connected_components_std", "node_connectivity_avg", "node_connectivity_std", "imp_time_avg", "imp_time_std", "imp_iterations_avg", "imp_iterations_std", "imp_bricks_avg", "imp_bricks_std", "imp_connected_components_avg", "imp_connected_components_std", "imp_node_connectivity_avg", "imp_node_connectivity_std"])
modelsdf = pd.DataFrame(columns=["model", "resolution", "volume", "voxels", "overhangs", "unsupported", "outside", "surface"])

with open("../out/results_compare.txt", "w") as log:
    
    for path in pathlist:
        mesh_name = str(path)
        print(mesh_name)

        mesh = trimesh.load(mesh_name)
        rot_angle = lego.optimal_rotation_angle(mesh)
        mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])

        for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium'), (1e8*.7,'high')]: 
            #mesh loading and voxelize
            mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
            voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
            outside = mesh.voxelized(pitch=single_lego_sizes).matrix
            #saving meshes and arrays
            trimesh.exchange.export.export_mesh(mesh, f"../out/meshes/{path.parts[-1]}_{resolution}.ply")
            trimesh.exchange.export.export_mesh(mesh.voxelized(pitch=single_lego_sizes).as_boxes(), f"../out/meshes/{path.parts[-1]}_{resolution}_blocks.ply")
            np.save(f"../out/ndarrays/{path.parts[-1]}_{resolution}_fill.npy", voxels, allow_pickle=False)
            np.save(f"../out/ndarrays/{path.parts[-1]}_{resolution}_out.npy", outside, allow_pickle=False)
            #loggin
            log.write(f"\n{path.parts[-1]} - resolution {resolution} - total voxels: {voxels.sum():,}")
            
            ## Statistics on the voxelization
            unsupported = 0
            for iz in range(0, voxels.shape[2]):
                for ix in range(voxels.shape[0]):
                    for iy in range(voxels.shape[1]):
                        if (voxels[ix,iy,iz] != 0) and (iz == 0) and (voxels[ix,iy,iz+1] == 0):
                            unsupported += 1
                            continue
                        if (voxels[ix,iy,iz] != 0) and (iz == voxels.shape[2]-1) and (voxels[ix,iy,iz-1]) == 0:
                            unsupported += 1
                            continue
                        if (voxels[ix,iy,iz] != 0) and (voxels[ix,iy,iz-1] == 0) and (voxels[ix,iy,iz+1]) == 0:
                            unsupported += 1
            overhangs = 0
            for ix in range(voxels.shape[0]):
                for iy in range(voxels.shape[1]):
                    overhang = False
                    for iz in range(1, voxels.shape[2]):
                        if voxels[ix,iy,iz] == 0:
                            overhang = True
                        elif overhang is True and voxels[ix,iy,iz] != 0:
                            overhangs += 1
            surface = 0
            for ix in range(voxels.shape[0]):
                for iy in range(voxels.shape[1]):
                    for iz in range(1, voxels.shape[2]):
                        if voxels[ix,iy,iz] != 0:
                            if ix == 0 or iy == 0 or iz == 0 or ix == voxels.shape[0]-1 or iy == voxels.shape[1]-1 or iz == voxels.shape[2]-1:
                                surface += 1
                            elif voxels[ix-1,iy,iz] == 0 or voxels[ix+1,iy,iz] or voxels[ix,iy-1,iz] == 0 or voxels[ix,iy+1,iz] == 0 or voxels[ix,iy,iz-1] == 0 or voxels[ix,iy,iz+1] == 0:
                                surface +=1
            modelsdf.loc[modelsdf.shape[0]] = [path.parts[-1], resolution, mesh.volume, int(voxels.sum()), overhangs, unsupported, outside.sum(), surface]
            
            ## Layout creation
            #creation of valid indices for the node connectivity test
            
            valid_voxels = np.argwhere(voxels)
            connectivity_indices = [valid_voxels[np.random.choice(valid_voxels.shape[0], 2, replace=False), :] for cp in range(num_connectivity_pair)]
            
            times = []
            num_nodes = []
            num_comps = []
            connectivity = []
            opt_tries = []
            opt_times = []
            opt_num_nodes = []
            opt_num_comps = []
            opt_connectivity = []
            for repeat in range(num_reapeat):
                start_time = time.time()
                lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=True)
                times.append(time.time() - start_time)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                num_nodes.append(graph.number_of_nodes())
                num_comps.append(nx.number_connected_components(conncomp_graph))
                connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))
                
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps[-1])
                opt_times.append(time.time() - start_time)
                opt_tries.append(tries)                
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                opt_num_nodes.append(graph.number_of_nodes())
                opt_num_comps.append(nx.number_connected_components(conncomp_graph))
                opt_connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))
            log.write(f"\n\n\tRandom executions")
            log.write(f"\n\tElapsed time: avg {np.mean(times):.3f} - std {np.std(times):.3f} seconds")
            log.write(f"\n\tNumber of bricks: avg {np.mean(num_nodes):.3f} - std {np.std(num_nodes):.3f}")
            log.write(f"\n\tNumber of connected components: avg {np.mean(num_comps):.3f} - std {np.std(num_comps):.3f}")
            log.write(f"\n\tNode connectivity on {num_connectivity_pair} pair of nodes: avg {np.mean(connectivity):.3f} - std {np.std(connectivity):.3f}")
            log.write(f"\n\tImprovement attempts")
            log.write(f"\n\tElapsed time: avg {np.mean(opt_times):.3f} - std {np.std(opt_times):.3f} seconds")
            log.write(f"\n\tIterations performed: avg {np.mean(opt_tries):.3f} - std {np.std(opt_tries):.3f}")
            log.write(f"\n\tNumber of bricks: avg {np.mean(opt_num_nodes):.3f} - std {np.std(opt_num_nodes):.3f}")
            log.write(f"\n\tNumber of connected components: avg {np.mean(opt_num_comps):.3f} - std {np.std(opt_num_comps):.3f}")
            log.write(f"\n\tNode connectivity on {num_connectivity_pair} pair of nodes: avg {np.mean(opt_connectivity):.3f} - std {np.std(opt_connectivity):.3f}")
            resultsdf.loc[resultsdf.shape[0]] = [path.parts[-1], resolution, "random", np.mean(times), np.std(times), np.mean(num_nodes), np.std(num_nodes),
                                                 np.mean(num_comps), np.std(num_comps), np.mean(connectivity), np.std(connectivity), np.mean(opt_times), 
                                                 np.std(opt_times), np.mean(opt_tries), np.std(opt_tries), np.mean(opt_num_nodes), np.std(opt_num_nodes), 
                                                 np.mean(opt_num_comps), np.std(opt_num_comps), np.mean(opt_connectivity), np.std(opt_connectivity)]
            
            times = []
            num_nodes = []
            num_comps = []
            connectivity = []
            opt_tries = []
            opt_times = []
            opt_num_nodes = []
            opt_num_comps = []
            opt_connectivity = []
            for repeat in range(num_reapeat):
                start_time = time.time()
                lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=False)
                times.append(time.time() - start_time)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                num_nodes.append(graph.number_of_nodes())
                num_comps.append(nx.number_connected_components(conncomp_graph))
                connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))
                
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps[-1])
                opt_times.append(time.time() - start_time)
                opt_tries.append(tries)                
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                opt_num_nodes.append(graph.number_of_nodes())
                opt_num_comps.append(nx.number_connected_components(conncomp_graph))
                opt_connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))
            log.write(f"\n\n\tEuristic executions")
            log.write(f"\n\tElapsed time: avg {np.mean(times):.3f} - std {np.std(times):.3f} seconds")
            log.write(f"\n\tNumber of bricks: avg {np.mean(num_nodes):.3f} - std {np.std(num_nodes):.3f}")
            log.write(f"\n\tNumber of connected components: avg {np.mean(num_comps):.3f} - std {np.std(num_comps):.3f}")
            log.write(f"\n\tNode connectivity on {num_connectivity_pair} pair of nodes: avg {np.mean(connectivity):.3f} - std {np.std(connectivity):.3f}")
            log.write(f"\n\tImprovement attempts")
            log.write(f"\n\tElapsed time: avg {np.mean(opt_times):.3f} - std {np.std(opt_times):.3f} seconds")
            log.write(f"\n\tIterations performed: avg {np.mean(opt_tries):.3f} - std {np.std(opt_tries):.3f}")
            log.write(f"\n\tNumber of bricks: avg {np.mean(opt_num_nodes):.3f} - std {np.std(opt_num_nodes):.3f}")
            log.write(f"\n\tNumber of connected components: avg {np.mean(opt_num_comps):.3f} - std {np.std(opt_num_comps):.3f}")
            log.write(f"\n\tNode connectivity on {num_connectivity_pair} pair of nodes: avg {np.mean(opt_connectivity):.3f} - std {np.std(opt_connectivity):.3f}")
            resultsdf.loc[resultsdf.shape[0]] = [path.parts[-1], resolution, "euristic", np.mean(times), np.std(times), np.mean(num_nodes), np.std(num_nodes),
                                                 np.mean(num_comps), np.std(num_comps), np.mean(connectivity), np.std(connectivity), np.mean(opt_times), 
                                                 np.std(opt_times), np.mean(opt_tries), np.std(opt_tries), np.mean(opt_num_nodes), np.std(opt_num_nodes), 
                                                 np.mean(opt_num_comps), np.std(opt_num_comps), np.mean(opt_connectivity), np.std(opt_connectivity)]
            
            log.write("\n")
            log.flush()


#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')

#resultsdf = resultsdf.apply(pd.to_numeric, errors='ignore')
#modelsdf = modelsdf.apply(pd.to_numeric, errors='ignore')

#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')

Utilitiesa for computing metrics and reassemble the layouts

In [ ]:
def compute_connectivity(lego_matrix_in, graph_in, connectivity_indices):
    lego_matrix = np.copy(lego_matrix_in)  
    graph = graph_in.copy()
    values = []
    id2node = [None,]
    for node in graph.nodes:
        id2node.append(node)
    for ci in connectivity_indices:
        values.append(nx.algorithms.connectivity.local_node_connectivity(graph, id2node[int(lego_matrix[tuple(ci[0])])], id2node[int(lego_matrix[tuple(ci[1])])]))
    return np.mean(values)

In [ ]:
def merge_connected_components(lego_matrix_in, graph_in, component_dictionary_in, max_tries=20):
    lego_matrix = np.copy(lego_matrix_in)  
    graph = graph_in.copy()
    if component_dictionary_in is None:
        component_dictionary = None
    else:
        component_dictionary = dict(component_dictionary_in)
    tries = 0
    while tries < max_tries:
        wx,wy,wz = 5,5,3
        if component_dictionary is None: # there is only one connected component
            break
        tries += 1
        aux = dict()
        for k,v in component_dictionary.items():
            aux.setdefault(v,[]).append(k)
        components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
        id2node = [None,]
        for node in graph.nodes:
            id2node.append(node)
        c = random.choice(components[1:])
        if not c: # if c is empty
            break
            
        wx,wy,wz = wx+random.randint(0,2),wy+random.randint(0,2),wz+random.randint(0,1)
        nodes = [graph.nodes[id2node[round(n)]] for n in  c[1]]
        #print(nodes)
        center = random.choice(nodes)["end"]
        graph = lego.disassemble_region(graph, lego_matrix, *center, wx,wy,wz)
        lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=False, graph_in=graph)
    return lego_matrix, component_dictionary, graph, tries

In [ ]:
def reassemle_articulation_points_old(lego_matrix_in, graph_in, max_tries=20):
    lego_matrix = np.copy(lego_matrix_in)  
    graph = graph_in.copy()
    tries = 0
    beginning_arts = None
    component_dictionary = None
    while beginning_arts is None or tries < max_tries + beginning_arts:
        conncomp_graph = graph.copy()
        db.see_conn_comp(conncomp_graph)
        arts = []
        art_p = list(nx.articulation_points(conncomp_graph))
        OG_components = list(nx.connected_components(conncomp_graph))
        aux_counter = 0
        G = conncomp_graph.copy()
        start_time = time.time()
        for pa in art_p:
            tempnode = conncomp_graph.nodes[pa]
            tempedge = conncomp_graph.edges(pa)
            G.remove_node(pa)
            gen_comps = nx.connected_components(G)
            gen_comps = [g for g in gen_comps if g not in OG_components]
            sizes = [len(c) for c in gen_comps]
            assert(len(sizes) >= 2)
            sizes.sort(reverse=True)
            if sizes[1] > 1:
                arts.append(pa)
            G.add_node(pa, **tempnode)
            G.add_edges_from(tempedge, weight = 1)
        #in arts are the proper articulation points
        #print(time.time()-start_time)
        if len(arts) == 0:
            break
        tries += 1
        if beginning_arts is None: 
            beginning_arts = len(arts)
        wx,wy,wz = 5,5,3
        wx,wy,wz = wx+random.randint(0,2),wy+random.randint(0,2),wz+random.randint(0,1)
        node = random.choice(arts)
        center = tuple( (node[i]+graph.nodes[node]['end'][i])//2 for i in range(3))
        graph = lego.disassemble_region(graph, lego_matrix, *center, wx,wy,wz)
        start_time = time.time()
        lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=False, graph_in=graph)
        #print(time.time()-start_time)
    return lego_matrix, component_dictionary, graph, tries

def reassemle_articulation_points(lego_matrix_in, graph_in, max_tries=20):
    lego_matrix = np.copy(lego_matrix_in)  
    graph = graph_in.copy()
    tries = 0
    beginning_arts = None
    component_dictionary = None
    conncomp_graph = graph.copy()
    db.see_conn_comp(conncomp_graph)
    og_components_number = nx.number_connected_components(conncomp_graph)
    while beginning_arts is None or tries < max_tries + beginning_arts:
        conncomp_graph = graph.copy()
        db.see_conn_comp(conncomp_graph)
        arts = []
        art_p = list(nx.articulation_points(conncomp_graph))
        G = conncomp_graph.copy()
        start_time = time.time()
        for pa in art_p:
            neigbors = conncomp_graph.neighbors(pa)
            tempnode = conncomp_graph.nodes[pa]
            tempedge = conncomp_graph.edges(pa)
            G.remove_node(pa)
            neigbors = [n for n in neigbors if len(G.edges(n)) > 0]
            if len(neigbors) <= 1:
                pass
            else:
                n0 = neigbors[0]            
                for n in neigbors[1:]:
                    if not nx.has_path(G, n0, n):
                        arts.append(pa)
                        break
            G.add_node(pa, **tempnode)
            G.add_edges_from(tempedge, weight = 1)
        #in arts are the proper articulation points
        #print(time.time()-start_time)
        if len(arts) == 0:
            break
        tries += 1
        if beginning_arts is None: 
            beginning_arts = len(arts)
        wx,wy,wz = 5,5,3
        wx,wy,wz = wx+random.randint(0,2),wy+random.randint(0,2),wz+random.randint(0,1)
        node = random.choice(arts)
        center = tuple( (node[i]+graph.nodes[node]['end'][i])//2 for i in range(3))
        graph_backup = graph.copy()
        graph = lego.disassemble_region(graph, lego_matrix, *center, wx,wy,wz)
        start_time = time.time()
        n_lego_matrix, n_component_dictionary, n_graph = lego.build(mesh, single_lego_sizes, use_random_priority=False, graph_in=graph)
        #connected_comps_graph = graph_backup.copy()
        #db.see_conn_comp(connected_comps_graph)
        #num_comps = nx.number_connected_components(connected_comps_graph)
        connected_comps_graph = n_graph.copy()
        db.see_conn_comp(connected_comps_graph)
        if nx.number_connected_components(connected_comps_graph) <= og_components_number:
            lego_matrix, component_dictionary, graph = n_lego_matrix, n_component_dictionary, n_graph
        else:
            graph = graph_backup.copy()
            continue
        #print(time.time()-start_time)
    return lego_matrix, component_dictionary, graph, tries

# Batch executions
From here this cells generate the results used in the paper.

In [ ]:
single_lego_sizes = np.array([8,8,9.6])
num_reapeat = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
componentsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "bricks_conn_comp_avg", "bricks_conn_comp_std", 
                                     "voxels_conn_comp_avg", "voxels_conn_comp_std", "outside_percentage_avg", "outside_percentage_std", 
                                     "imp_bricks_conn_comp_avg", "imp_bricks_conn_comp_std", 
                                     "imp_voxels_conn_comp_avg", "imp_voxels_conn_comp_std", "imp_outside_percentage_avg", "imp_outside_percentage_std"])
    
for path in pathlist:
    mesh_name = str(path)
    print(mesh_name)

    mesh = trimesh.load(mesh_name)
    rot_angle = lego.optimal_rotation_angle(mesh)
    mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])

    for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium'), (1e8*.7,'high')]: 
        #mesh loading and voxelize
        mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
        voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
        outside = mesh.voxelized(pitch=single_lego_sizes).matrix

        ## Layout creation
        #creation of valid indices for the node connectivity test

        bricks_conn_comp = []
        voxels_conn_comp = []
        outside_percentage = []
        opt_bricks_conn_comp = []
        opt_voxels_conn_comp = []
        opt_outside_percentage = []
        for repeat in range(num_reapeat):
            lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=True)
            conncomp_graph = graph.copy()
            db.see_conn_comp(conncomp_graph)
            num_comps = nx.number_connected_components(conncomp_graph)
            if num_comps > 1:
                id2node = [None,]
                for node in graph.nodes:
                    id2node.append(node)
                aux = dict()
                for k,v in component_dictionary.items():
                    aux.setdefault(v,[]).append(k)
                components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)[1:]
                components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
                components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
                c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ c for (a,b,c) in components]
                bricks_conn_comp.append(np.mean(c1))
                voxels_conn_comp.append(np.mean(c2))
                outside_percentage.append(1-np.mean(c3))
            else:
                bricks_conn_comp.append(0)
                voxels_conn_comp.append(0)
                outside_percentage.append(1)
            
            lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps)
            conncomp_graph = graph.copy()
            db.see_conn_comp(conncomp_graph)
            num_comps = nx.number_connected_components(conncomp_graph)
            if num_comps > 1:
                id2node = [None,]
                for node in graph.nodes:
                    id2node.append(node)
                aux = dict()
                for k,v in component_dictionary.items():
                    aux.setdefault(v,[]).append(k)
                components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)[1:]
                components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
                components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
                c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ c for (a,b,c) in components]
                opt_bricks_conn_comp.append(np.mean(c1))
                opt_voxels_conn_comp.append(np.mean(c2))
                opt_outside_percentage.append(1-np.mean(c3))
            else:
                opt_bricks_conn_comp.append(0)
                opt_voxels_conn_comp.append(0)
                opt_outside_percentage.append(1)
            
        componentsdf.loc[componentsdf.shape[0]] = [path.parts[-1], resolution, "random", np.mean(bricks_conn_comp), np.std(bricks_conn_comp), 
                                                np.mean(voxels_conn_comp), np.std(voxels_conn_comp), np.mean(outside_percentage), np.std(outside_percentage),
                                                np.mean(opt_bricks_conn_comp), np.std(opt_bricks_conn_comp), 
                                                np.mean(opt_voxels_conn_comp), np.std(opt_voxels_conn_comp), 
                                                np.mean(opt_outside_percentage), np.std(opt_outside_percentage)]

        bricks_conn_comp = []
        voxels_conn_comp = []
        outside_percentage = []
        opt_bricks_conn_comp = []
        opt_voxels_conn_comp = []
        opt_outside_percentage = []
        for repeat in range(num_reapeat):
            lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=False)
            conncomp_graph = graph.copy()
            db.see_conn_comp(conncomp_graph)
            num_comps = nx.number_connected_components(conncomp_graph)
            num_comps = nx.number_connected_components(conncomp_graph)
            if num_comps > 1:
                id2node = [None,]
                for node in graph.nodes:
                    id2node.append(node)
                aux = dict()
                for k,v in component_dictionary.items():
                    aux.setdefault(v,[]).append(k)
                components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)[1:]
                components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
                components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
                c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ c for (a,b,c) in components]
                bricks_conn_comp.append(np.mean(c1))
                voxels_conn_comp.append(np.mean(c2))
                outside_percentage.append(1-np.mean(c3))
            else:
                bricks_conn_comp.append(0)
                voxels_conn_comp.append(0)
                outside_percentage.append(1)
            
            lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps)
            conncomp_graph = graph.copy()
            db.see_conn_comp(conncomp_graph)
            num_comps = nx.number_connected_components(conncomp_graph)
            if num_comps > 1:
                id2node = [None,]
                for node in graph.nodes:
                    id2node.append(node)
                aux = dict()
                for k,v in component_dictionary.items():
                    aux.setdefault(v,[]).append(k)
                components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)[1:]
                components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
                components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
                c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ c for (a,b,c) in components]
                opt_bricks_conn_comp.append(np.mean(c1))
                opt_voxels_conn_comp.append(np.mean(c2))
                opt_outside_percentage.append(1-np.mean(c3))
            else:
                opt_bricks_conn_comp.append(0)
                opt_voxels_conn_comp.append(0)
                opt_outside_percentage.append(1)
        componentsdf.loc[componentsdf.shape[0]] = [path.parts[-1], resolution, "euristic", np.mean(bricks_conn_comp), np.std(bricks_conn_comp), 
                                                np.mean(voxels_conn_comp), np.std(voxels_conn_comp), np.mean(outside_percentage), np.std(outside_percentage),
                                                np.mean(opt_bricks_conn_comp), np.std(opt_bricks_conn_comp), 
                                                np.mean(opt_voxels_conn_comp), np.std(opt_voxels_conn_comp), 
                                                np.mean(opt_outside_percentage), np.std(opt_outside_percentage)]
        componentsdf.to_csv('../out/components.csv')

#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')
componentsdf.to_csv('../out/components.csv')

#resultsdf = resultsdf.apply(pd.to_numeric, errors='ignore')
#modelsdf = modelsdf.apply(pd.to_numeric, errors='ignore')
componentsdf = componentsdf.apply(pd.to_numeric, errors='ignore')

#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')
componentsdf.to_csv('../out/components.csv')


In [ ]:
#bests 

single_lego_sizes = np.array([8,8,9.6])
num_repeat = 10
num_connectivity_pair = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
bestsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "times", "bricks", "connected_components", "node_connectivity", "best_id",
                                "imp_times", "imp_iterations", "imp_bricks", "imp_connected_components", "imp_node_connectivity", "imp_best_id"])
bestcomponentsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "bricks_conn_comps", "voxels_conn_comps", "component_outside",
                                         "imp_bricks_conn_comps", "imp_voxels_conn_comps", "imp_component_outside"])

   
for path in pathlist:
    mesh_name = str(path)
    print(mesh_name)

    mesh = trimesh.load(mesh_name)
    rot_angle = lego.optimal_rotation_angle(mesh)
    mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])

    for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium'), (1e8*.7,'high')]: 
        #mesh loading and voxelize
        mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
        voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
        outside = mesh.voxelized(pitch=single_lego_sizes).matrix
        print(voxels.shape)
        #saving meshes and arrays

        ## Layout creation
        #creation of valid indices for the node connectivity test

        valid_voxels = np.argwhere(voxels)
        connectivity_indices = [valid_voxels[np.random.choice(valid_voxels.shape[0], 2, replace=False), :] for cp in range(num_connectivity_pair)]

        for random_method in [True, False]:
            string_method = 'random'
            if random_method == False:
                string_method = 'euristic'
            times = []
            num_nodes = []
            num_comps = []
            connectivity = []
            opt_tries = []
            opt_times = []
            opt_num_nodes = []
            opt_num_comps = []
            opt_connectivity = []
            lego_matrixes = []
            component_dictionaries = []
            graphs = []
            opt_lego_matrixes = []
            opt_component_dictionaries = []
            opt_graphs = []
            for repeat in range(num_repeat):
                start_time = time.time()
                lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=random_method)
                times.append(time.time() - start_time)
                lego_matrixes.append(lego_matrix)
                component_dictionaries.append(component_dictionary)
                graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                num_nodes.append(graph.number_of_nodes())
                num_comps.append(nx.number_connected_components(conncomp_graph))
                connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))

            best_id = 0
            for i in range(num_repeat):
                if (num_comps[best_id] > num_comps[i]) or ((num_comps[best_id] == num_comps[i]) and (num_nodes[best_id] > num_nodes[i])):
                    best_id = i

            for repeat in range(num_repeat):
                lego_matrix = lego_matrixes[best_id]
                component_dictionary = component_dictionaries[best_id]
                graph = graphs[best_id]
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps[-1])
                opt_times.append(time.time() - start_time)
                opt_tries.append(tries)
                opt_lego_matrixes.append(lego_matrix)
                opt_component_dictionaries.append(component_dictionary)
                opt_graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                opt_num_nodes.append(graph.number_of_nodes())
                opt_num_comps.append(nx.number_connected_components(conncomp_graph))
                opt_connectivity.append(compute_connectivity(lego_matrix, conncomp_graph, connectivity_indices))
                
            opt_best_id = 0
            for i in range(num_repeat):
                if (opt_num_comps[opt_best_id] > opt_num_comps[i]) or \
                ((opt_num_comps[opt_best_id] == opt_num_comps[i]) and (opt_num_nodes[opt_best_id] > opt_num_nodes[i])):
                    opt_best_id = i
            bestsdf.loc[bestsdf.shape[0]] = [path.parts[-1], resolution, string_method, times, num_nodes, num_comps, connectivity, best_id,
                                             opt_times, opt_tries, opt_num_nodes, opt_num_comps, opt_connectivity, opt_best_id]
            bestsdf.to_csv('../out/bests.csv')

            bricks_conn_comp = []
            voxels_conn_comp = []
            outside_percentage = []
            opt_bricks_conn_comp = []
            opt_voxels_conn_comp = []
            opt_outside_percentage = []
            
            lego_matrix = lego_matrixes[best_id]
            component_dictionary = component_dictionaries[best_id]
            graph = graphs[best_id]
            id2node = [None,]
            for node in graph.nodes:
                id2node.append(node)
            aux = dict()
            if component_dictionary is None:
                component_dictionary = {a:0 for a in range(1, num_nodes[best_id]+1)}
            for k,v in component_dictionary.items():
                aux.setdefault(v,[]).append(k)
            components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
            components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
            components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
            c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ not c for (a,b,c) in components]
            bricks_conn_comp = c1
            voxels_conn_comp = c2
            outside_percentage = c3
            lego_matrix = opt_lego_matrixes[opt_best_id]
            component_dictionary = opt_component_dictionaries[opt_best_id]
            graph = opt_graphs[opt_best_id]
            id2node = [None,]
            for node in graph.nodes:
                id2node.append(node)
            aux = dict()
            if component_dictionary is None:
                component_dictionary = {a:0 for a in range(1, len(id2node))}
            for k,v in component_dictionary.items():
                aux.setdefault(v,[]).append(k)
            components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
            components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
            components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
            c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ not c for (a,b,c) in components]
            opt_bricks_conn_comp = c1
            opt_voxels_conn_comp = c2
            opt_outside_percentage = c3
            bestcomponentsdf.loc[bestcomponentsdf.shape[0]] = [path.parts[-1], resolution, string_method, bricks_conn_comp, voxels_conn_comp, outside_percentage,
                                                               opt_bricks_conn_comp, opt_voxels_conn_comp, opt_outside_percentage]
            bestcomponentsdf.to_csv('../out/bestcomponents.csv')
                
#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')
bestsdf.to_csv('../out/bests.csv')
bestcomponentsdf.to_csv('../out/bestcomponents.csv')

#resultsdf = resultsdf.apply(pd.to_numeric, errors='ignore')
#modelsdf = modelsdf.apply(pd.to_numeric, errors='ignore')

#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')


In [ ]:
#articulation points 

single_lego_sizes = np.array([8,8,9.6])
num_repeat = 10
num_connectivity_pair = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
articulationsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "bad_articulation_points", "opt_bad_articulation_points"])

   
for path in pathlist:
    mesh_name = str(path)
    print(mesh_name)

    mesh = trimesh.load(mesh_name)
    rot_angle = lego.optimal_rotation_angle(mesh)
    mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])

    for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium'), (1e8*.7,'high')]: 
        #mesh loading and voxelize
        mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
        voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
        outside = mesh.voxelized(pitch=single_lego_sizes).matrix
        print(voxels.shape)
        #saving meshes and arrays

        for random_method in [True, False]:
            string_method = 'random'
            if random_method == False:
                string_method = 'euristic'
            times = []
            num_nodes = []
            num_comps = []
            connectivity = []
            opt_tries = []
            opt_times = []
            opt_num_nodes = []
            opt_num_comps = []
            opt_connectivity = []
            lego_matrixes = []
            component_dictionaries = []
            graphs = []
            opt_lego_matrixes = []
            opt_component_dictionaries = []
            opt_graphs = []
            
            arts = []
            opt_arts = []

            for repeat in range(num_repeat):
                start_time = time.time()
                lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=random_method)
                times.append(time.time() - start_time)
                lego_matrixes.append(lego_matrix)
                component_dictionaries.append(component_dictionary)
                graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                num_nodes.append(graph.number_of_nodes())
                num_comps.append(nx.number_connected_components(conncomp_graph))

                art_p = list(nx.articulation_points(conncomp_graph))
                OG_components = list(nx.connected_components(conncomp_graph))
                #print((list(art_p), len(list(art_p))))
                aux_counter = 0
                G = conncomp_graph.copy()
                for pa in art_p:
                    tempnode = conncomp_graph.nodes[pa]
                    tempedge = conncomp_graph.edges(pa)
                    G.remove_node(pa)
                    gen_comps = nx.connected_components(G)
                    gen_comps = [g for g in gen_comps if g not in OG_components]
                    sizes = [len(c) for c in gen_comps]
                    assert(len(sizes) >= 2)
                    sizes.sort(reverse=True)
                    if sizes[1] > 1:
                        aux_counter += 1
                    G.add_node(pa, **tempnode)
                    G.add_edges_from(tempedge, weight = 1)
                        #print((sizes,pa,conncomp_graph.nodes[pa]))
                #print((len(OG_components), len(art_p), aux_counter))
                arts.append(aux_counter)

            best_id = 0
            for i in range(num_repeat):
                if (num_comps[best_id] > num_comps[i]) or ((num_comps[best_id] == num_comps[i]) and (num_nodes[best_id] > num_nodes[i])):
                    best_id = i

            for repeat in range(num_repeat):
                lego_matrix = lego_matrixes[best_id]
                component_dictionary = component_dictionaries[best_id]
                graph = graphs[best_id]
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps[-1])
                opt_times.append(time.time() - start_time)
                opt_tries.append(tries)
                opt_lego_matrixes.append(lego_matrix)
                opt_component_dictionaries.append(component_dictionary)
                opt_graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                opt_num_nodes.append(graph.number_of_nodes())
                opt_num_comps.append(nx.number_connected_components(conncomp_graph))
                
                art_p = list(nx.articulation_points(conncomp_graph))
                OG_components = list(nx.connected_components(conncomp_graph))
                aux_counter = 0
                G = conncomp_graph.copy()
                for pa in art_p:
                    tempnode = conncomp_graph.nodes[pa]
                    tempedge = conncomp_graph.edges(pa)
                    G.remove_node(pa)
                    gen_comps = nx.connected_components(G)
                    gen_comps = [g for g in gen_comps if g not in OG_components]
                    sizes = [len(c) for c in gen_comps]
                    assert(len(sizes) >= 2)
                    sizes.sort(reverse=True)
                    if sizes[1] > 1:
                        aux_counter += 1
                    G.add_node(pa, **tempnode)
                    G.add_edges_from(tempedge, weight = 1)
                opt_arts.append(aux_counter)
                
            opt_best_id = 0
            for i in range(num_repeat):
                if (opt_num_comps[opt_best_id] > opt_num_comps[i]) or \
                ((opt_num_comps[opt_best_id] == opt_num_comps[i]) and (opt_num_nodes[opt_best_id] > opt_num_nodes[i])):
                    opt_best_id = i
            articulationsdf.loc[articulationsdf.shape[0]] = [path.parts[-1], resolution, string_method, arts, opt_arts]
            #articulationsdf.to_csv('../out/articulationsdf.csv')
            
articulationsdf.to_csv('../out/articulationsdf.csv')



In [ ]:
#bests + articulations

single_lego_sizes = np.array([8,8,9.6])
num_repeat = 10
num_connectivity_pair = 10
pathlist = Path("../meshes/dataset/").glob("*.ply")
bestartsdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "times", "bricks", "connected_components", "articulation_points", "best_id",
                                "imp_times", "imp_iterations", "imp_bricks", "imp_connected_components", "imp_articulation_points", "imp_best_id",
                                "art_times", "art_iterations", "art_bricks", "art_connected_components", "art_articulation_points", "art_best_id"])
bestcomponentsartdf = pd.DataFrame(columns=["model", "resolution", "execution_type", "bricks_conn_comps", "voxels_conn_comps", "component_outside",
                                         "imp_bricks_conn_comps", "imp_voxels_conn_comps", "imp_component_outside",
                                         "art_bricks_conn_comps", "art_voxels_conn_comps", "art_component_outside"])

   
for path in pathlist:
    mesh_name = str(path)
    print(mesh_name)

    mesh = trimesh.load(mesh_name)
    rot_angle = lego.optimal_rotation_angle(mesh)
    mesh = utils.rotate_mesh(mesh, -rot_angle, [0,0,1])

    for target_volume, resolution in [(1e6*.7,'low'), (1e7*.7,'medium')]:#, (1e8*.7,'high')]: 
        #mesh loading and voxelize
        mesh = utils.fit_mesh_to_volume(mesh, target_volume, single_lego_sizes[0])
        voxels = mesh.voxelized(pitch=single_lego_sizes).fill().matrix
        outside = mesh.voxelized(pitch=single_lego_sizes).matrix
        print(voxels.shape)
        #saving meshes and arrays


        for random_method in [True, False]:
            string_method = 'random'
            if random_method == False:
                string_method = 'euristic'
            times = []
            num_nodes = []
            num_comps = []
            articulations = []
            opt_tries = []
            opt_times = []
            opt_num_nodes = []
            opt_num_comps = []
            opt_articulations = []
            art_tries = []
            art_times = []
            art_num_nodes = []
            art_num_comps = []
            art_articulations = []
            lego_matrixes = []
            component_dictionaries = []
            graphs = []
            opt_lego_matrixes = []
            opt_component_dictionaries = []
            opt_graphs = []
            art_lego_matrixes = []
            art_component_dictionaries = []
            art_graphs = []
            for repeat in range(num_repeat):
                start_time = time.time()
                lego_matrix, component_dictionary, graph = lego.build(mesh, single_lego_sizes, use_random_priority=random_method)
                times.append(time.time() - start_time)
                lego_matrixes.append(lego_matrix)
                component_dictionaries.append(component_dictionary)
                graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                num_nodes.append(graph.number_of_nodes())
                num_comps.append(nx.number_connected_components(conncomp_graph))
                #articulations
                art_p = list(nx.articulation_points(conncomp_graph))
                OG_components = list(nx.connected_components(conncomp_graph))
                aux_counter = 0
                G = conncomp_graph.copy()
                for pa in art_p:
                    tempnode = conncomp_graph.nodes[pa]
                    tempedge = conncomp_graph.edges(pa)
                    G.remove_node(pa)
                    gen_comps = nx.connected_components(G)
                    gen_comps = [g for g in gen_comps if g not in OG_components]
                    sizes = [len(c) for c in gen_comps]
                    assert(len(sizes) >= 2)
                    sizes.sort(reverse=True)
                    if sizes[1] > 1:
                        aux_counter += 1
                    G.add_node(pa, **tempnode)
                    G.add_edges_from(tempedge, weight = 1)
                articulations.append(aux_counter)

            best_id = 0
            for i in range(num_repeat):
                if (num_comps[best_id] > num_comps[i]):
                    best_id = i
                elif (num_comps[best_id] == num_comps[i]) and (articulations[best_id] > articulations[i]):
                    best_id = i
                elif ((num_comps[best_id] == num_comps[i]) and (articulations[best_id] == articulations[i])) and (num_nodes[best_id] > num_nodes[i]):
                    best_id = i

            for repeat in range(num_repeat):
                lego_matrix = lego_matrixes[best_id]
                component_dictionary = component_dictionaries[best_id]
                graph = graphs[best_id]
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = merge_connected_components(lego_matrix, graph, component_dictionary, 20+num_comps[-1])
                opt_times.append(time.time() - start_time)
                opt_tries.append(tries)
                opt_lego_matrixes.append(lego_matrix)
                opt_component_dictionaries.append(component_dictionary)
                opt_graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                opt_num_nodes.append(graph.number_of_nodes())
                opt_num_comps.append(nx.number_connected_components(conncomp_graph))
                #articulations
                art_p = list(nx.articulation_points(conncomp_graph))
                OG_components = list(nx.connected_components(conncomp_graph))
                aux_counter = 0
                G = conncomp_graph.copy()
                for pa in art_p:
                    tempnode = conncomp_graph.nodes[pa]
                    tempedge = conncomp_graph.edges(pa)
                    G.remove_node(pa)
                    gen_comps = nx.connected_components(G)
                    gen_comps = [g for g in gen_comps if g not in OG_components]
                    sizes = [len(c) for c in gen_comps]
                    assert(len(sizes) >= 2)
                    sizes.sort(reverse=True)
                    if sizes[1] > 1:
                        aux_counter += 1
                    G.add_node(pa, **tempnode)
                    G.add_edges_from(tempedge, weight = 1)
                opt_articulations.append(aux_counter)
                
            opt_best_id = 0
            for i in range(num_repeat):
                if (opt_num_comps[opt_best_id] > opt_num_comps[i]):
                    opt_best_id = i
                elif (opt_num_comps[opt_best_id] == opt_num_comps[i]) and (opt_articulations[opt_best_id] > opt_articulations[i]):
                    opt_best_id = i
                elif ((opt_num_comps[opt_best_id] == opt_num_comps[i]) and (opt_articulations[opt_best_id] == opt_articulations[i])) \
                      and (opt_num_nodes[opt_best_id] > opt_num_nodes[i]):
                    opt_best_id = i
                    
            
            for repeat in range(num_repeat):
                lego_matrix = opt_lego_matrixes[opt_best_id]
                component_dictionary = opt_component_dictionaries[opt_best_id]
                graph = opt_graphs[opt_best_id]
                start_time = time.time()
                lego_matrix, component_dictionary, graph, tries = reassemle_articulation_points(lego_matrix, graph, opt_articulations[opt_best_id]) #this doubles the max tries
                art_times.append(time.time() - start_time)
                art_tries.append(tries)
                art_lego_matrixes.append(lego_matrix)
                art_component_dictionaries.append(component_dictionary)
                art_graphs.append(graph)
                conncomp_graph = graph.copy()
                db.see_conn_comp(conncomp_graph)
                art_num_nodes.append(graph.number_of_nodes())
                art_num_comps.append(nx.number_connected_components(conncomp_graph))
                #articulations
                art_p = list(nx.articulation_points(conncomp_graph))
                OG_components = list(nx.connected_components(conncomp_graph))
                aux_counter = 0
                G = conncomp_graph.copy()
                for pa in art_p:
                    tempnode = conncomp_graph.nodes[pa]
                    tempedge = conncomp_graph.edges(pa)
                    G.remove_node(pa)
                    gen_comps = nx.connected_components(G)
                    gen_comps = [g for g in gen_comps if g not in OG_components]
                    sizes = [len(c) for c in gen_comps]
                    assert(len(sizes) >= 2)
                    sizes.sort(reverse=True)
                    if sizes[1] > 1:
                        aux_counter += 1
                    G.add_node(pa, **tempnode)
                    G.add_edges_from(tempedge, weight = 1)
                art_articulations.append(aux_counter)
                
            art_best_id = 0
            for i in range(num_repeat):
                if (art_num_comps[art_best_id] > art_num_comps[i]):
                    art_best_id = i
                elif (art_num_comps[art_best_id] == art_num_comps[i]) and (art_articulations[art_best_id] > art_articulations[i]):
                    art_best_id = i
                elif ((art_num_comps[art_best_id] == art_num_comps[i]) and (art_articulations[art_best_id] == art_articulations[i])) \
                      and (art_num_nodes[art_best_id] > art_num_nodes[i]):
                    art_best_id = i
                    
                    
            
            bestartsdf.loc[bestartsdf.shape[0]] = [path.parts[-1], resolution, string_method, times, num_nodes, num_comps, articulations, best_id,
                                                opt_times, opt_tries, opt_num_nodes, opt_num_comps, opt_articulations, opt_best_id,
                                                art_times, art_tries, art_num_nodes, art_num_comps, art_articulations, art_best_id]
            bestartsdf.to_csv('../out/bestarts.csv')

            bricks_conn_comp = []
            voxels_conn_comp = []
            outside_percentage = []
            opt_bricks_conn_comp = []
            opt_voxels_conn_comp = []
            opt_outside_percentage = []
            
            art_bricks_conn_comp = []
            art_voxels_conn_comp = []
            art_outside_percentage = []
            
            lego_matrix = lego_matrixes[best_id]
            component_dictionary = component_dictionaries[best_id]
            graph = graphs[best_id]
            id2node = [None,]
            for node in graph.nodes:
                id2node.append(node)
            aux = dict()
            if component_dictionary is None:
                component_dictionary = {a:0 for a in range(1, num_nodes[best_id]+1)}
            for k,v in component_dictionary.items():
                aux.setdefault(v,[]).append(k)
            components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
            components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
            components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
            c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ not c for (a,b,c) in components]
            bricks_conn_comp = c1
            voxels_conn_comp = c2
            outside_percentage = c3
            
            lego_matrix = opt_lego_matrixes[opt_best_id]
            component_dictionary = opt_component_dictionaries[opt_best_id]
            graph = opt_graphs[opt_best_id]
            id2node = [None,]
            for node in graph.nodes:
                id2node.append(node)
            aux = dict()
            if component_dictionary is None:
                component_dictionary = {a:0 for a in range(1, len(id2node))}
            for k,v in component_dictionary.items():
                aux.setdefault(v,[]).append(k)
            components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
            components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
            components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
            c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ not c for (a,b,c) in components]
            opt_bricks_conn_comp = c1
            opt_voxels_conn_comp = c2
            opt_outside_percentage = c3
            
            lego_matrix = art_lego_matrixes[art_best_id]
            component_dictionary = art_component_dictionaries[art_best_id]
            graph = art_graphs[art_best_id]
            id2node = [None,]
            for node in graph.nodes:
                id2node.append(node)
            aux = dict()
            if component_dictionary is None:
                component_dictionary = {a:0 for a in range(1, len(id2node))}
            for k,v in component_dictionary.items():
                aux.setdefault(v,[]).append(k)
            components = sorted(list(aux.items()), key=lambda x: len(x[1]), reverse = True)
            components = [ [ (graph.nodes[id2node[e]]['inner'], lego.get_node_shape(id2node[e], graph).replace('x','*')) for e in v] for (k,v) in components]
            components = [ (len(c), np.sum([eval(b) for (a,b) in c]), np.prod([a for (a,b) in c])) for c in components]
            c1,c2,c3 = [ a for (a,b,c) in components],[ b for (a,b,c) in components],[ not c for (a,b,c) in components]
            art_bricks_conn_comp = c1
            art_voxels_conn_comp = c2
            art_outside_percentage = c3
            
            bestcomponentsartdf.loc[bestcomponentsartdf.shape[0]] = [path.parts[-1], resolution, string_method, bricks_conn_comp, voxels_conn_comp, outside_percentage,
                                                                    opt_bricks_conn_comp, opt_voxels_conn_comp, opt_outside_percentage,
                                                                    art_bricks_conn_comp, art_voxels_conn_comp, art_outside_percentage]
            bestcomponentsartdf.to_csv('../out/bestcomponentsart.csv')
                
#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')
bestartsdf.to_csv('../out/bestarts.csv')
bestcomponentsartdf.to_csv('../out/bestcomponentsart.csv')

#resultsdf = resultsdf.apply(pd.to_numeric, errors='ignore')
#modelsdf = modelsdf.apply(pd.to_numeric, errors='ignore')

#resultsdf.to_csv('../out/results.csv')
#modelsdf.to_csv('../out/models.csv')
